# sveltejs/svelte

> 

In [4]:
#| default_exp sveltejs-svelte

In [5]:
#| hide
import nbdev; nbdev.nbdev_export()

In [6]:
#| export
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import logging

In [7]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
#| export

source =  GitSource(
    name="sveltejs/svelte",
    doc_dir="apps/svelte.dev/content/docs/svelte",
    repo="https://github.com/sveltejs/svelte.dev",
    commit="main",
    ecosystems=["svelte"])


commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

#### Read the markdown files

In [ ]:
#| export

tree = build_markdown_doc_tree(clone_dir/source.doc_dir)

#### Process the tree

In [7]:
# tree.subdirs = tree.subdirs[:2] # For testing process just the first 2 secstion (intro and runes)

In [8]:
#| export

processed_tree = await process_tree_depth_first(settings, tree, source.name)

+0.884s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 01-introduction/01-overview.md
+0.302s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 01-introduction/02-getting-started.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 01-introduction/03-svelte-files.md
+0.015s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 01-introduction/04-svelte-js-files.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 01-introduction/index.md
+0.019s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 02-runes/01-what-are-runes.md
+0.020s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 02-runes/02-$state.md
+0.018s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 02-runes/03-$derived.md
+0.018s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 02-runes/04-$effect.md
+0.021s DEBUG lovely_docs/docs.py:128 llm_process_page Processing 02-runes/05-$props.md
+0.018s DEBUG lovely_docs/docs.py:128 llm_process_pa

#### Calculate cost

In [9]:
usage = calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 354,324
  Output tokens: 92,944
  Total tokens: 447,268
Cost:
  Total: $0.82
  Input: $0.35
  Output: $0.46


#### save the results

In [10]:
#| export

save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)